In [4]:
import cv2 as cv
import numpy as np

In [5]:
imgWidth = 640
imgHeight = 480
cap = cv.VideoCapture(0)
cap.set(3, imgWidth)
cap.set(4, imgHeight)
cap.set(10, 150)

True

### Preprocessing

In [6]:
def preProcessing(img):
    imgGray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    imgBlur = cv.GaussianBlur(imgGray, (5,5), 1)
    imgCanny = cv.Canny(imgBlur, 200, 200) # finding edges
    kernel = np.ones((5,5))
    imgDial = cv.dilate(imgCanny, kernel, iterations=2)
    imgThresh = cv.erode(imgDial, kernel, iterations=1)
    
    return imgThresh

### Getting the Contours

In [17]:
def getContours(img):
    biggest = np.array([])
    maxArea = 0
    contours, hierarchy = cv.findContours(img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE) # finds all the contours
    for cnt in contours: # loops through each contour
        area = cv.contourArea(cnt)
        if area > 500: # setting a threshold for the contour area
            peri = cv.arcLength(cnt, True)
            approx = cv.approxPolyDP(cnt, 0.02*peri, True) # corner points
            if area > maxArea and len(approx) == 4: # checks if it has 4 sides (rect/square) and the area is the maximum
                biggest = approx
                maxArea = area
    cv.drawContours(imgContour, biggest, -1, (255, 0, 0), 20)
    return biggest

### Reordering

**The corner points received from the above function isn't necessarily in the order required.**

**Therefore, we need to reorder them into the right format**

In [8]:
def reorder (myPoints):
    myPoints = myPoints.reshape((4,2)) # we reshape to use numpy functions
    myPointsNew = np.zeros((4,1,2),np.int32) # the output has to be the same shape as the input
    add = myPoints.sum(1)
    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] = myPoints[np.argmax(add)]
    diff = np.diff(myPoints, axis=1)
    myPointsNew[1]= myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
    return myPointsNew

### Warping

In [9]:
def getWarp(img, biggest):
    biggest = reorder(biggest)
    pts1 = np.float32(biggest) # the actual position of the corner points
    pts2 = np.float32([[0, 0], [imgWidth, 0], [0, imgHeight], [imgWidth, imgHeight]]) # the positions into which we want to map our image
    matrix = cv.getPerspectiveTransform(pts1, pts2)
    imgOutput = cv.warpPerspective(img, matrix, (imgWidth, imgHeight))

    imgCropped = imgOutput[20:imgOutput.shape[0]-20, 20:imgOutput.shape[1]-20] # removes extra portion
    imgCropped = cv.resize(imgCropped, (imgWidth, imgHeight))

    return imgCropped

### Stacking the images to be shown

In [14]:
def stackImages(scale, imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv.cvtColor( imgArray[x][y], cv.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv.cvtColor(imgArray[x], cv.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

In [19]:
while True:
    success, img = cap.read()
    img = cv.resize(img, (imgWidth, imgHeight))
    imgContour = img.copy()

    imgThres = preProcessing(img)
    biggest = getContours(imgThres)
    if biggest.size !=0:
        imgWarped=getWarp(img,biggest)
        imageArray = ([img,imgThres],
                   [imgContour,imgWarped])
        cv.imshow("ImageWarped", imgWarped)
    else:
        imageArray = ([img, imgThres],
                       [img, img])

    stackedImages = stackImages(0.6, imageArray)
    cv.imshow("WorkFlow", stackedImages)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break